In [ ]:
import numpy as np
import plotly.graph_objects as go
from pickle import dump
from datetime import datetime
from quantfreedom import dl_ex_candles
from quantfreedom.indicators.tv_indicators import ema_tv

%load_ext autoreload
%autoreload 2

In [ ]:
pkl_file_path = "D:\TDBot\candle_databases\pickle_files\candles.pkl"

candles = dl_ex_candles(
    exchange = "mufex",
    symbol = "BTCUSDT",
    timeframe = "5m",
    since_datetime = datetime(2023, 10, 11),
    until_datetime = datetime(2023, 12, 5),
)

In [ ]:
with open(pkl_file_path,"wb") as f:
    dump(candles,f)

In [ ]:
first_ema_length = 50
second_ema_length = 140
candle_close_prices = candles.candle_close_prices
datetimes = candles.candle_open_datetimes

In [ ]:
first_ema_not_rounded = ema_tv(
    source=candle_close_prices,
    length=first_ema_length,
)

first_ema = np.around(first_ema_not_rounded, decimals=2)

first_ema_p = np.roll(first_ema,1)
first_ema_p[0]= np.nan
print(first_ema)
print(first_ema_p)

In [ ]:
second_ema_not_rounded = ema_tv(
    source=candle_close_prices,
    length=second_ema_length,
)

second_ema = np.around(second_ema_not_rounded, decimals=2)

second_ema_p = np.roll(second_ema,1)
second_ema_p[0]= np.nan
print(second_ema)
print(second_ema_p)

In [ ]:
cur_first_above_second = first_ema > second_ema
prev_first_below_second = first_ema_p < second_ema

entries = cur_first_above_second & prev_first_below_second

entry_signals = np.where (entries, first_ema, np.nan)

exit_price = np.full_like(first_ema, np.nan)

In [ ]:
fig = go.Figure()
fig.add_candlestick(
    x=datetimes,
    open=candles.candle_open_prices,
    high=candles.candle_high_prices,
    low=candles.candle_low_prices,
    close=candle_close_prices,
    name="candles",
)
fig.add_scatter(
    x=datetimes,
    y=first_ema,
    name="First EMA",
    line=dict(color="yellow",width=3),
)
fig.add_scatter(
    x=datetimes,
    y=second_ema,
    name="second EMA",
    line=dict(color="blue",width=3),
)
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    name="Buy Signal",
    mode="markers",
    marker=dict(size=8, symbol="circle",color="#00F6FF"),
)
fig.update_layout(height=600, xaxis_rangeslider_visible=False)
fig.show()